In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import radians, log10
from scipy.io import loadmat
import json
import seaborn as sns
import pvlib
import glob
import re
from matplotlib.colors import Normalize

In [3]:
# PV Solar Cell Parameters
Tref = 25  # Reference temperature（℃）
eta_ref = 0.20  # Reference power generation efficiency
c1, c2, c3 = -3.75, 1.14, 0.0175
rou = 0.0045  # Temperature effect coefficient
sigma = 0.1  # Solar irradiance adjustment coefficient

In [4]:
# solar position
As_list_pd = pd.read_excel('solar azimuth angle.xlsx')
As_list = As_list_pd['Solar\nAzimuth:'].to_list()
hs_list = As_list_pd['Solar\nElevation:'].to_list()
solar_x, solar_y, solar_z = As_list_pd['solar_pos_x'].astype(float).to_list(), As_list_pd['solar_pos_y'].astype(float).to_list(), As_list_pd['solar_pos_z'].astype(float).to_list()

In [5]:
epwfile = pvlib.iotools.read_epw('USA_CA_San.Francisco-Presidio.994016_TMYx.2009-2023/USA_CA_San.Francisco-Presidio.994016_TMYx.2009-2023.epw', coerce_year=2022)[0]
epwfile_june13 = epwfile[(epwfile['month']==6) & (epwfile['day']==13)]
epwfile_june13[['temp_air']].to_csv('SF_airtemp.csv',index=False)

In [6]:
# Climate data for SF
#############################################################################################
# Read solar radiation data
Rt_h = epwfile_june13[['ghi']].values.flatten() # Total (global) solar radiation on the horizontal surface
Rs_h = epwfile_june13[['dhi']].values.flatten() # Diffuse Horizontal Irradiance 
Rd_h = Rt_h - Rs_h                              # Direct Horizontal Irradiance
# Rd_h = epwfile_june13[['dni']].values.flatten() # this is the normal surface, not horizontal

# Read ambient temperature data
T = pd.read_csv('SF_airtemp.csv').values.flatten() #Ambient temperature
print(T.shape)
# Downsampling
# Rt_h = Rt_h[::60]
# Rd_h = Rd_h[::60]
# Rs_h = Rs_h[::60]
# T = T[::60]
# The above solar radiation intensity and ambient temperature are real San Francisco data
#############################################################################################

(24,)


In [ ]:
# test_case = 'Chinatown' 
test_case = 'Inner Richmond'  

vector_simulation = pd.read_csv(f'../results/results for paper/{test_case}_vector.csv', header=0)

vector_simulation = vector_simulation.astype(float)
if len(vector_simulation) < 24:
    rows_to_add = 24 - len(vector_simulation)
    nan_rows = pd.DataFrame(np.nan, index=range(rows_to_add), columns=vector_simulation.columns)
    vector_simulation = pd.concat([vector_simulation, nan_rows], ignore_index=True)
vector_simulation
for col in vector_simulation.columns:
    parts = col.split('_')
    if parts[0] == 'roof': 
        value_to_fill = float(parts[2])
    else:
        value_to_fill = float(parts[3])
    vector_simulation[col] = vector_simulation[col].fillna(value_to_fill)
vector_simulation = vector_simulation.apply(lambda x: pd.to_numeric(x, errors='coerce')).round()

In [329]:
pd.set_option("display.max_columns", 56)

In [330]:
vector_simulation_lit_area = vector_simulation.copy()
columns_to_process = vector_simulation_lit_area.columns
for col in columns_to_process:
    vector_simulation_lit_area[col] = vector_simulation_lit_area[col].max() - vector_simulation_lit_area[col]
vector_simulation_lit_area = vector_simulation_lit_area.T
vector_simulation_lit_area

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
roof_0_1500,0.0,0.0,0.0,0.0,0.0,299.0,182.0,1367.0,1483.0,1435.0,1474.0,1475.0,1478.0,1475.0,1499.0,1500.0,1500.0,1500.0,1372.0,1150.0,0.0,0.0,0.0,0.0
roof_1_2648,0.0,0.0,0.0,0.0,0.0,2648.0,2648.0,2648.0,2648.0,2648.0,2648.0,2648.0,2648.0,2648.0,2648.0,2648.0,2648.0,2648.0,2632.0,1856.0,0.0,0.0,0.0,0.0
roof_2_600,0.0,0.0,0.0,0.0,0.0,0.0,600.0,600.0,600.0,600.0,600.0,600.0,600.0,600.0,600.0,587.0,508.0,600.0,600.0,453.0,0.0,0.0,0.0,0.0
roof_3_221,0.0,0.0,0.0,0.0,0.0,211.0,221.0,221.0,221.0,221.0,221.0,221.0,221.0,221.0,221.0,221.0,221.0,221.0,188.0,139.0,0.0,0.0,0.0,0.0
roof_4_804,0.0,0.0,0.0,0.0,0.0,414.0,442.0,693.0,773.0,790.0,793.0,797.0,796.0,804.0,804.0,804.0,804.0,804.0,804.0,804.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67_12965_10_1270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1246.0,1261.0,1268.0,1174.0,1074.0,1004.0,1004.0,0.0,0.0,0.0,0.0
67_12965_12_644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,644.0,644.0,644.0,586.0,539.0,509.0,509.0,0.0,0.0,0.0,0.0
67_12965_14_10479,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10479.0,10479.0,10479.0,9636.0,8835.0,8279.0,8279.0,0.0,0.0,0.0,0.0
67_12965_16_227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,194.0,201.0,218.0,216.0,192.0,181.0,179.0,0.0,0.0,0.0,0.0


In [ ]:
# load facade azimuth
file_pattern = f'../results/{test_case}_results_building_loc_original_code_time*.json'
matching_files = glob.glob(file_pattern)
matching_files
with open(matching_files[0], 'r') as f:
    building_data = json.load(f)
def get_facade_azimuths(building_id, index):
    for cell in building_data.values():  # Iterate over cell values directly
        for building_key, building in cell.items():
            if re.match(rf"^{building_id}_building_\d+$", building_key):
                facade_azimuth = building.get('facade_azimuth', index)
                if "roof" not in index.lower():
                    return facade_azimuth[index]

# Create a dictionary to store facade azimuths for each building
facade_azimuth_dicts = {}
for index, row in vector_simulation_lit_area.iterrows():
    building_id = extract_building_id(index)
    azimuth = get_facade_azimuths(building_id, index)
    facade_azimuth_dicts[index] = azimuth

In [332]:
def extract_building_id(index_name):
    parts = index_name.split('_')
    if parts[0] == 'roof':
        return parts[1]
    else:
        return parts[0]
def determine_beta(index_name):
    return 0 if index_name.startswith('roof') else 90
def determine_azimuth(index_name):
    parts = index_name.split('_')
    if parts[0] == 'roof':
        return 0
    else:
        return parts[0]

# Create the building_ID column
vector_simulation_lit_area['building_ID'] = vector_simulation_lit_area.index.map(extract_building_id)
vector_simulation_lit_area['beta'] = vector_simulation_lit_area.index.map(determine_beta)
vector_simulation_lit_area['azimuth'] = vector_simulation_lit_area.index.map(facade_azimuth_dicts.get)
# vector_simulation_lit_area['azimuth'] = vector_simulation_lit_area['azimuth'].fillna(0)
# vector_simulation_lit_area['azimuth'] = vector_simulation_lit_area['azimuth'] + 90
vector_simulation_lit_area

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,building_ID,beta,azimuth
roof_0_1500,0.0,0.0,0.0,0.0,0.0,299.0,182.0,1367.0,1483.0,1435.0,1474.0,1475.0,1478.0,1475.0,1499.0,1500.0,1500.0,1500.0,1372.0,1150.0,0.0,0.0,0.0,0.0,0,0,NaN
roof_1_2648,0.0,0.0,0.0,0.0,0.0,2648.0,2648.0,2648.0,2648.0,2648.0,2648.0,2648.0,2648.0,2648.0,2648.0,2648.0,2648.0,2648.0,2632.0,1856.0,0.0,0.0,0.0,0.0,1,0,NaN
roof_2_600,0.0,0.0,0.0,0.0,0.0,0.0,600.0,600.0,600.0,600.0,600.0,600.0,600.0,600.0,600.0,587.0,508.0,600.0,600.0,453.0,0.0,0.0,0.0,0.0,2,0,NaN
roof_3_221,0.0,0.0,0.0,0.0,0.0,211.0,221.0,221.0,221.0,221.0,221.0,221.0,221.0,221.0,221.0,221.0,221.0,221.0,188.0,139.0,0.0,0.0,0.0,0.0,3,0,NaN
roof_4_804,0.0,0.0,0.0,0.0,0.0,414.0,442.0,693.0,773.0,790.0,793.0,797.0,796.0,804.0,804.0,804.0,804.0,804.0,804.0,804.0,0.0,0.0,0.0,0.0,4,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67_12965_10_1270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1246.0,1261.0,1268.0,1174.0,1074.0,1004.0,1004.0,0.0,0.0,0.0,0.0,67,90,267.296706
67_12965_12_644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,644.0,644.0,644.0,586.0,539.0,509.0,509.0,0.0,0.0,0.0,0.0,67,90,267.665491
67_12965_14_10479,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10479.0,10479.0,10479.0,9636.0,8835.0,8279.0,8279.0,0.0,0.0,0.0,0.0,67,90,267.246433
67_12965_16_227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,194.0,201.0,218.0,216.0,192.0,181.0,179.0,0.0,0.0,0.0,0.0,67,90,267.776716


Rt_h # Total (global) solar radiation on the horizontal surface 

Rs_h # Diffuse Horizontal Irradiance  

Rd_h # Direct Horizontal Irradiance 

In [333]:
# PV calculation, required parameters
#############################################################################################
# facade_azimuth = 180  #  Facade azimuth angle: South=180, East=90, West=-90, North=0
# beta = 90  # Facade angle (tilt), 90 degrees means vertical to the ground
# unshaded_area = np.full(24, 40) #  Unshaded area of facade in square meters, should be a vector. Set as constant in this code
#############################################################################################
# Define functions to calculate cosine and tangent (using degrees)
def cosd(angle):
    return np.cos(np.deg2rad(angle))

def tand(angle):
    return np.tan(np.deg2rad(angle))

def calc_PVs_24h(row, row_index):
    facade_azimuth = row['azimuth']
    beta = row['beta']
    unshaded_area = row[0:24].values 
    Ppv_cals = []
    for i in range(24):
        if "roof" not in row_index:
            As = As_list[i]
            hs = hs_list[i]
            # Calculating Solar Radiation Components on a Tilted Surface
            cos_theta = cosd(facade_azimuth - As)
            Rd = Rd_h[i] * cos_theta / tand(hs) if hs > 0 else 0
            Rr = 0.15 * Rt_h[i] * (1 - np.cos(radians(beta))) / 2
            Rs = Rs_h[i] * (1 + np.cos(radians(beta))) / 2
            Rt = Rd + Rs + Rr        
        else:
            Rt = Rt_h[i]
        # Solar PV Power Generation Calculation
        # print(f"==== {i} ===== ")
        # print(Rt)
        Ppv = Rt * eta_ref * (1 - rou * (c1 + c2 * T[i] + c3 * Rt - Tref) + sigma * np.log10(Rt))  if Rt > 0 else 0
        Ppv_cal = unshaded_area[i] * Ppv
        Ppv_cals.append(Ppv_cal)
    return Ppv_cals

In [334]:
pv_values = vector_simulation_lit_area.apply(
    lambda row: calc_PVs_24h(row, row.name),
    axis=1
)
for i in range(24):
    vector_simulation_lit_area[f'pv_{i}'] = pv_values.apply(lambda x: x[i])

In [335]:
vector_simulation_lit_area.to_csv(f'PV_{test_case}.csv')

In [13]:
roof_data = vector_simulation_lit_area[vector_simulation_lit_area.index.str.contains('roof')]
facade_data = vector_simulation_lit_area[~vector_simulation_lit_area.index.str.contains('roof')]

In [322]:
print('MW')
roof_pv_sum = roof_data[[col for col in roof_data.columns if str(col).startswith('pv')]].sum().sum()/1e6
facade_pv_sum = facade_data[[col for col in facade_data.columns if str(col).startswith('pv')]].sum().sum()/1e6
print(f'roof {roof_pv_sum}')
print(f'facade {facade_pv_sum}')

MW
roof 547.897066961664
facade 1734.2473088570189
